# Import libraries

In [2]:
import pandas as pd
from torch.utils.data import DataLoader
from dataset import NCFDataset
from recom_ncf import NCFRecommender
from evaluation import Evaluation
from helpers.index_manager import IndexManager
from helpers.splitter import Splitter
from helpers.dataloader_custom_functions import collate_fn

# Loading data

In [3]:
df = pd.read_csv('../data/interaction-clean.csv')[['user_id', 'item_id', 'rating_imp', 'timestamp']]
df.head()

,user_id,item_id,rating_imp,timestamp
0,76561197960432447,10,1,1738278781
1,76561198071230926,10,1,1736206418
2,76561198206216352,10,1,1738041574
3,76561198110801124,10,1,1738015332
4,76561199813732773,10,1,1737853720


# Indexing data

In [4]:
index_manager = IndexManager()
index_manager.fit(df_interaction=df)
index_manager.transform_interactions(df, inplace=True)

Indexed 836887 users and 69001 items
User index range: 0-836886
Item index range: 0-69000


,user_id,item_id,rating_imp,timestamp
0,0,0,1,1738278781
1,1,0,1,1736206418
2,2,0,1,1738041574
3,3,0,1,1738015332
4,4,0,1,1737853720
...,...,...,...,...
1156221,15849,24891,1,1663545142
1156222,150857,24891,1,1689826252
1156223,1336,24892,1,1595699739
1156224,11584,24892,0,1706470035


# Creating datasets

In [5]:
splitter = Splitter(df)
df_train, df_val, df_test = splitter.leave_k_out_split()

Splitting data with leave-2-out strategy (1 for validation, 1 for testing)
Total users: 836887
Interactions per user: min=1, max=1035, avg=1.4
Note: 706515 users have fewer than 2 interactions.
These users will be placed entirely in the training set.
Split complete: 1156226 total interactions
Train set: 895482 interactions (77.4%)
Validation set: 130372 interactions (11.3%)
Test set: 130372 interactions (11.3%)


In [6]:
train_dataset = NCFDataset(df_train)
val_dataset = NCFDataset(df_val)

dataloader_params = {
    'batch_size': 2**12,
    # 'num_workers': 4,
    # 'persistent_workers': True,
    # 'prefetch_factor': 4,
    'pin_memory': True,
    'pin_memory_device': 'cuda',
    'collate_fn': collate_fn,
}

train_dataloader = DataLoader(train_dataset, shuffle=True, **dataloader_params)
eval_dataloader = DataLoader(val_dataset, shuffle=False, **dataloader_params)

# Training model

In [7]:
params = {'factors': 8, 'mlp_user_item_dim': 128, 'learning_rate': 0.005, 'epochs': 5, 'optimizer': 'adagrad', 'dropout': 0.0, 'weight_decay': 0.0001, 'loss_fn': 'mse'}

unique_users = index_manager.get_indexed_users()
unique_items = index_manager.get_indexed_items()

model = NCFRecommender(unique_users, unique_items, **params)
model.fit(train_dataloader, eval_dataloader)

Epoch 1/5
Train loss: 0.140880, Validation loss: 0.141613
Epoch 2/5
Train loss: 0.113580, Validation loss: 0.142883
Epoch 3/5
Train loss: 0.041731, Validation loss: 0.166845
Epoch 4/5
Train loss: 0.030197, Validation loss: 0.167814
Early stopping triggered after 4 epochs
Training completed!


# Evaluating model

In [8]:
evaluator = Evaluation(recommender=model, test_data=df_test)
metrics = evaluator.evaluate()

Creating ground truth sets...
Generating predictions...
Processing 1 of 130372 users... (0.00%)
Memory usage: 0.292724609375 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 128 to 141
Increased item batch size from 1024 to 1126
Processing 129 of 130372 users... (0.10%)
Memory usage: 0.181884765625 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 141 to 155
Increased item batch size from 1126 to 1239
Processing 270 of 130372 users... (0.21%)
Memory usage: 0.19677734375 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 155 to 170
Increased item batch size from 1239 to 1363
Processing 425 of 130372 users... (0.33%)
Memory usage: 0.2119140625 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 170 to 187
Increased item batch size from 1363 to 1499
Processing 595 of 130372 users... (0.46%)
Memory usage: 0.229736328125 . Increasing batch size with increasing ra

KeyboardInterrupt: 

In [8]:
metrics

{'Hit Ratio@10': 0.011014635044334673,
 'NDCG@10': 0.008220008934469699,
 'Recall@10': 0.011014635044334673}